In [1]:
import numpy as np
from testCasesGC import *
from gc_utils import sigmoid,relu,dictionary_to_vector,vector_to_dictionary,gradients_to_vector

In [2]:
def forward_propogation(x,theta):
    j=theta*x
    
    return j

In [3]:
x,theta=2,4
j=forward_propogation(x,theta)
print('J='+str(j))

J=8


In [4]:
def backward_propagation(x,theta):
    dtheta=x
    return dtheta

In [5]:
x,theta=2,4
dtheta=backward_propagation(x,theta)
print('dtheta='+str(dtheta))

dtheta=2


In [6]:
def gradient_check(x,theta,epsilon=1e-7,print_msg=False):
    theta_plus=theta+epsilon
    theta_minus=theta-epsilon
    j_plus=forward_propogation(x,theta_plus)
    j_minus=forward_propogation(x,theta_minus)
    gradapprox=(j_plus-j_minus)/(2*epsilon)
    
    grad=backward_propagation(x,theta)
    
    numerator=np.linalg.norm(grad-gradapprox)
    denominator=np.linalg.norm(grad)+np.linalg.norm(gradapprox)
    difference=numerator/denominator
    
    if print_msg:
        if difference>2e-7:
            print('There is  a mistake in the back prop difference'+str(difference))
        else:
            print('back prop works fine differnce'+str(difference))
            
    return difference

In [7]:
#testing if this works
x,theta=2,4
difference=gradient_check(2,4,print_msg=True)

back prop works fine differnce2.919335883291695e-10


In [8]:
def forward_propagation_n(X,Y,parameters):
    m=X.shape[1]
    W1=parameters['W1']
    b1=parameters['b1']
    W2=parameters['W2']
    b2=parameters['b2']
    W3=parameters['W3']
    b3=parameters['b3']
    
    Z1=np.dot(W1,X)+b1
    A1=relu(Z1)
    Z2=np.dot(W2,A1)+b2
    A2=relu(Z2)
    Z3=np.dot(W3,A2)+b3
    A3=sigmoid(Z3)
    
    log_probs=np.multiply(-np.log(A3),Y)+np.multiply(-np.log(1-A3),1-Y)
    cost=1./m*np.sum(log_probs)
    
    cache=(Z1,A1,W1,b1,Z2,A2,W2,b2,Z3,A3,W3,b3)
    
    return cost,cache

In [17]:
def backward_propagation_n(X,Y,cache):
    m=X.shape[1]
    (Z1,A1,W1,b1,Z2,A2,W2,b2,Z3,A3,W3,b3)=cache
    
    dZ3=A3-Y
    dW3=1./m*np.dot(dZ3,A2.T)
    db3=1./m*np.sum(dZ3,axis=1,keepdims=True)
    
    dA2=np.dot(W3.T,dZ3)
    dZ2=np.multiply(dA2,np.int64(A2>0))
    dW2=1./m*np.dot(dZ2,A1.T)*2 #mistake here *2 not required
    
    #dW2=1./m*np.dot(dZ2,A1.T) #without mistake
    
    db2=1./m*np.sum(dZ2,axis=1,keepdims=True)
    
    dA1=np.dot(W2.T,dZ2)
    dZ1=np.multiply(dA1,np.int64(A1>0))
    dW1=1./m*np.dot(dZ1,X.T)
    db1=1./m*np.sum(dZ1,axis=1,keepdims=True)
    
    
    gradients={'dZ3':dZ3,'dW3':dW3,'db3':db3,'dA2':dA2,
              'dZ2':dZ2,'dW2':dW2,'db2':db2,'dA1':dA1,
              'dZ1':dZ1,'dW1':dW1,'db1':db1}
    return gradients

> Gradient check -Check any errors ...

In [18]:
def gradient_check_n(parameters,gradient_check_nradients,X,Y,epsilon=1e-7,print_msg=False):
    
    parameters_values,_=dictionary_to_vector(parameters)
    
    grads=gradients_to_vector(gradients)
    num_parameters=parameters_values.shape[0]
    j_plus=np.zeros((num_parameters,1))
    j_minus=np.zeros((num_parameters,1))
    gradapprox=np.zeros((num_parameters,1))
    
    
    for i in range(num_parameters):
        theta_plus=np.copy(parameters_values)
        theta_plus[i][0]=theta_plus[i][0]+epsilon
        j_plus[i],_=forward_propagation_n(X,Y,vector_to_dictionary(theta_plus))
        
        theta_minus=np.copy(parameters_values)
        theta_minus[i][0]=theta_minus[i][0]-epsilon
        j_minus[i],_=forward_propagation_n(X,Y,vector_to_dictionary(theta_minus))
        
        gradapprox[i]=(j_plus[i]-j_minus[i])/(2*epsilon)
        
        
    numerator=np.linalg.norm(grads-gradapprox)
    denominator=np.linalg.norm(grads)+np.linalg.norm(gradapprox)
    difference=numerator/denominator
    
    if print_msg:
        if difference>2e-7:
            print('There is  a mistake in the back prop difference'+str(difference))
        else:
            print('back prop works fine differnce'+str(difference))
            
    return difference

In [19]:
X,Y,parameters=gradient_check_n_test_case()

cost,cache=forward_propagation_n(X,Y,parameters)
gradients=backward_propagation_n(X,Y,cache)
difference=gradient_check_n(parameters,gradients,X,Y,1e-7,True)
expected_values=[0.2850931567761623,1.189091302422996e-07]

There is  a mistake in the back prop difference0.17344196871198458


In [20]:
## shows the mistake in these code ...by gradient checking